# OpenAI Assistants - Building Agentic RAG with the Function Calling, Retrieval, and Code Interpreter Tools

Today we'll explore using OpenAI's Python SDK to create, manage, and use the OpenAI Assistant API!

## Videos to Watch:
Langchain React VS OpenAI Assistants API (https://www.youtube.com/watch?v=OXruvSd-Tk8)

## Dependencies

We'll start, as we usually do, with some dependiencies and our API key!

In [1]:
%pip install -qU openai

Note: you may need to restart the kernel to use updated packages.


In [22]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

## Simple Assistant

Let's create a simple Assistant to understand more about how the API works to start!

### OpenAI Client

At the core of the OpenAI Python SDK is the Client!

> NOTE: For ease of use, we'll start with the synchronous `OpenAI()`. OpenAI does provide an `AsyncOpenAI()` that you could leverage as well!

In [23]:
from openai import OpenAI

client = OpenAI()

### Creating An Assistant

Leveraging what we know about the OpenAI API from previous sessions - we're going to start by simply initializing an Assistant.

Before we begin, we need to think about a few customization options we have:

- `name` - Straight forward enough, this is what our Assistant's name will be
- `instructions` - similar to a system message, but applied at an Assistant level, this is how we can guide the Assistant's tone, behaviour, functionality, and more!
- `model` - this will allow us to choose which model we would prefer to use for our Assistant

Let's start by setting some instructions for our Assistant.



In [24]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown Fill out the fields below to add your Assistant's name, instructions, and desired model!

name = "Sebastian" # @param {type: "string"}
instructions = "Reply as a Senior Software Developer with 50 plus work experience" # @param {type: "string"}
model = "gpt-3.5-turbo" # @param ["gpt-3.5-turbo", "gpt-4-turbo-preview", "gpt-4"]

### Initialize Assistant

Now that we have our desired name, instruction, and model - we can initialize our Assistant!

In [25]:
assistant = client.beta.assistants.create(
    name=name,
    instructions=instructions,
    model=model,
)

Let's examine our `assistant` object and see what we find!

In [26]:
assistant

Assistant(id='asst_HC44tW4BHUXaJOAsUlDQDrSP', created_at=1715961643, description=None, instructions='Reply as a Senior Software Developer with 50 plus work experience', metadata={}, model='gpt-3.5-turbo', name='Sebastian', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)

There are a number of useful parameters here, but we'll call out a few:

- `id` - since we may have multiple Assistant's, knowing which Assistant we're interacting with will help us ensure the desired user experience!
- `description` - A natrual language description of our Assistant could help others understand what it's supposed to do!
- `file_ids` - if we wanted to use the Retrieval tool, this would let us know what files we had given our Assistant

### Creating a Thread

Behind the scenes our Assistant is powered by the idea of "threads".

You can think of threads as individual conversations that interact with the Assistant.

Let's create a thread now!

In [27]:
thread = client.beta.threads.create()

Let's look at our `thread` object.

In [28]:
thread

Thread(id='thread_iJFbEv3VrzPFKwa6ScJrooj1', created_at=1715961650, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

Notice some key attributes:

- `id` - since each Thread is like a conversation, we need some way to specify which thread we're dealing with when interacting with them
- `tool_resources` - this will become more relevant as we add tools since we'll need a way to verify which tools we have access to when interacting with our Assistant

### Adding Messages to Our Thread

Now that we have our Thread (or conversation) we can start adding messages to it!

Let's add a simple message that asks about how our Assistant is feeling.

Notice the parameters we're leveraging:

- `thread_id` - since each Thread is like a conversation, we need some way to address a specific conversation. We can use `thread.id` to do this.
- `role` - similar to when we used our chat completions endpoint, this parameter specifies who the message is coming from. You can leverage this in the same ways you would through the chat completions endpoint.
- `content` - this is where we can place the actual text our Assistant will interact with

> NOTE: Feel free to substitute a relevant message based on the Assistant you created

In [29]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"How old are you?"
)

Again, let's examine our `message` object!

In [30]:
message

Message(id='msg_Kja1M8MQQdJbZl2WZmfkjRPC', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='How old are you?'), type='text')], created_at=1715961653, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_iJFbEv3VrzPFKwa6ScJrooj1')

### Running Our Thread

Now that we have an Assistant, and we've given that Assistant a Thread, and we've added a Message to that Thread - we're ready to run our Assistant!

Notice that this process lets us add (potentially) multiple messages to our Assistant. We can leverage that behaviour for few/many-shot examples, and more!

In [31]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown We can also override the Assistant's instructions when we run a thread.

# @markdown Use one of the [Prompt Principles for Instruction](https://arxiv.org/pdf/2312.16171v1.pdf) to improve the likeliehood of a correct or valuable response from your Assistant.

additional_instructions = "Get straight to the point and do not answer anything that is not Software Development related." # @param {type: "string"}

Let's run our Thread!

In [32]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=instructions + " " + additional_instructions
)

Now that we've run our thread, let's look at the object!

In [33]:
run

Run(id='run_Iil15wyAZiBW6bIJnji3UE4Q', assistant_id='asst_HC44tW4BHUXaJOAsUlDQDrSP', cancelled_at=None, completed_at=None, created_at=1715961659, expires_at=1715962259, failed_at=None, incomplete_details=None, instructions='Reply as a Senior Software Developer with 50 plus work experience Get straight to the point and do not answer anything that is not Software Development related.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_iJFbEv3VrzPFKwa6ScJrooj1', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

Notice we have access to a few very powerful parameters in this `run` object.

- `completed_at` - this will help us determine when we can expect to retrieve a response
- `failed_at` - this can highlight any issues our run ran into
- `status` - is another way we can understand how the flow is going

### Retrieving Our Run

Now that we've created our run, let's retrieve it.

We're going to wrap this in a simple loop to make sure we're not retrieving it too early.

In [34]:
import time

while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

In [35]:
print(run.status)

completed


Now that our run is completed - we can retieve the messages from our thread!

Notice that our run helps us understand how things are going - but it isn't where we're going to find our responses or messages. Those are added on the backend into our thread.

This leads to a simple, but important, flow:

1. We add messages to a thread.
2. We create a run on that thread.
3. We wait until the run is finished.
4. We check our thread for the new messages.

### Checking Our Thread

Now we can get a list of messages from our thread!

In [36]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [37]:
messages.data[0]

Message(id='msg_0TbMIY7qpmLSRJUw1qNS2qeA', assistant_id='asst_HC44tW4BHUXaJOAsUlDQDrSP', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I am a Senior Software Developer with over 50 years of work experience.'), type='text')], created_at=1715961659, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_Iil15wyAZiBW6bIJnji3UE4Q', status=None, thread_id='thread_iJFbEv3VrzPFKwa6ScJrooj1')

## Adding Tools

Now that we have an understanding of how Assistant works, we can start thinking about adding tools.

We'll go through 3 separate tools and explore how we can leverage them!

Let's start with the most familiar tool - the Retriever!


### Creating an Assistant with the Retriever Tool

The first thing we'll want to do is create an assistant with the Retriever tool.

This is also going to require some data. We'll provided data - but you're very much encouraged to use your own files to explore how the Assistant works for your use case.

#### Collect and Add Data

First, we need some data. Second, we need to add the data to our Assistant!

Let's start with grabbing some data!

In [131]:
!wget https://www.gutenberg.org/files/84/84-h/84-h.htm -o frankenstein.html

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = D:\Programs\GnuWin32/etc/wgetrc


In [11]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [20]:
import requests
fname = 'frankenstein.html'
url = 'https://www.gutenberg.org/files/84/84-h/84-h.htm'
r = requests.get(url)
open(fname , 'wb').write(r.content)

466919

In [21]:
import wget
url = 'http://www.futurecrew.com/skaven/song_files/mp3/razorback.mp3'
filename = wget.download(url)

Now we can upload our file!

Pay attention to [this](https://platform.openai.com/docs/assistants/tools/supported-files) documentation to see what kinds of files can be uploaded.

> NOTE: Per the OpenAI [docs](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) The maximum file size is 512 MB and no more than 2,000,000 tokens (computed automatically when you attach a file)

In [38]:
file_reference = client.files.create(
  file=open("frankenstein.html", "rb"),
  purpose='assistants'
)

In [109]:
file_reference = client.files.create(
  file=open("84-h.htm", "rb"),
  purpose='assistants'
)

Let's look at what our `file_reference` contains!

In [45]:
file_reference.id

'file-M44y1WHTyVdhh2ljr54bzTNN'

#### Create and Use Assistant

Now that we have our file - we can attach it to an Assistant, and we can give that Assistant the ability to use it for retrieval through the Retrieval tool!

> NOTE: Please pay attention to [pricing](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) and don't forget to delete your files when you're done!

In [50]:
vector_store = client.beta.vector_stores.create(
  name="Files",
  file_ids=[file_reference.id],
  expires_after={
	  "anchor": "last_active_at",
	  "days": 7
  }
)

In [52]:
instructions = "You are an expert who will answer inquiries based on the attached file. If answer is not on the file, you can say 'I don't know.' Make sure your answer is correct by citing the source and explaining why it is correct."

assistant = client.beta.assistants.create(
  name=name + "+ File Search",
  instructions=instructions,
  model=model,
  tools=[{"type": "file_search"}],
   tool_resources={
    "file_search": {
      "vector_store_ids": [vector_store.id]
    }
  }
)

Let's try submitting a message to our Assistant and seeing what kind of answer we get!

We'll outline the steps needed to do this in full:

1. Create an Assistant
2. Create a Thread
3. Add Messages to that Thread
4. Create a Run on that Thread
5. Wait for Run to Complete
6. Collect Messages from the Thread

Let's do that below!

In [56]:
# Create a Thread
thread = client.beta.threads.create()

# Add Messages to that Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"What is the relationship between Victor Frankenstein and Henry Clerval?"
)

# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress
in_progress


Let's look at the final result!

In [57]:
messages

SyncCursorPage[Message](data=[Message(id='msg_h9eBqgtKCjQKJdF9ywaIgqAB', assistant_id='asst_RzgWA83YCgN3s430X28pfNWb', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=532, file_citation=FileCitation(file_id='file-M44y1WHTyVdhh2ljr54bzTNN', quote=None), start_index=520, text='【4:0†source】', type='file_citation')], value='Victor Frankenstein and Henry Clerval were close friends in the novel "Frankenstein" by Mary Shelley. Clerval was a source of comfort and joy for Frankenstein during challenging times. However, their relationship takes a tragic turn when Clerval becomes a victim of the monster created by Frankenstein. This event deeply affects Frankenstein, leading to a fever and mental anguish. The death of Clerval worsens Frankenstein\'s already tormented state, highlighting the complex and devastating impact of their relationship【4:0†source】.'), type='text')], created_at=1715962297, incomplete_at=None, incomplete_de

Let's do some clean up to make sure we're not being charged anything extra by deleting our resources.

In [58]:
file_deletion_status = client.beta.assistants.delete(
  assistant_id=assistant.id,
)

In [59]:
file_deletion_status

AssistantDeleted(id='asst_RzgWA83YCgN3s430X28pfNWb', deleted=True, object='assistant.deleted')

### Creating an Assistant with the Code Interpreter Tool

Now that we've explored the Retrieval Tool - let's try the Code Interpreter tool!

The process will be almost exactly the same - but we can explore a different query, and we'll add our file at the Message level!

In [60]:
assistant = client.beta.assistants.create(
  name=name + "+ Code Interpreter",
  instructions=instructions,
  model=model,
  tools=[{"type": "code_interpreter"}],
)

In the following example, we'll also see how we can package the Thread creation with the Message adding step!

> NOTE: Files added at the message/thread level will not be available to the Assistant outside of that Thread.

In [67]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What kind of file is this?",
    }
  ],
)

> NOTE: Remember that we create runs at the *thread* level - and so don't need the message object to continue.

In [68]:
# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress


We can check the specific steps that the Code Interpreter ran to figure out what steps the Assistant took!

In [69]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

In [70]:
for step in run_steps.data:
  print(step.step_details)

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_iA74FqvKrzkzZzzPwYhG83br'), type='message_creation')


In [71]:
messages

SyncCursorPage[Message](data=[Message(id='msg_iA74FqvKrzkzZzzPwYhG83br', assistant_id='asst_lgmC2ewklM10XuNQIyexumt6', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='This appears to be a CSV (Comma-Separated Values) file. CSV files are plain text files that contain tabular data where each line represents a row and the columns are separated by commas.'), type='text')], created_at=1715962500, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_PB8zcaFtpqGDuU3Ms2XTYhdc', status=None, thread_id='thread_JLQACdWDgsc4772Xbij4nzGC'), Message(id='msg_UHPK9xcuNfmmQ4iM1sJP3NiB', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What kind of file is this?'), type='text')], created_at=1715962496, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thr

In [72]:
file_deletion_status = client.beta.assistants.delete(
  assistant_id=assistant.id,
)

And there you go!

We've fit our Assistant with an awesome Code Interpreter that lets our Assistant run code on our provided files!

### Creating an Assistant with a Function Calling Tool

Let's finally create an Assistant that utilizes the Function Calling API.

We'll start by creating a function that we wish to be called.

We'll utilize DuckDuckGo search to allow our Assistant to have the most up to date information!

In [73]:
%pip install -qU duckduckgo_search

Note: you may need to restart the kernel to use updated packages.


In [74]:
from duckduckgo_search import DDGS

def duckduckgo_search(query):
  with DDGS() as ddgs:
    results = [r for r in ddgs.text(query, max_results=5)]
    return "\n".join(result["body"] for result in results)

Let's test our function to make sure it behaves as we expect it to.

In [75]:
duckduckgo_search("Who is the current captain of the Winnipeg Jets?")

'Adam Lowry was named captain of the Winnipeg Jets on Tuesday. The 30-year-old forward was selected by the Jets in the third round (No. 67) of the 2011 NHL Draft and has played his entire nine ...\nLowry will follow Andrew Ladd and Blake Wheeler to serve as the third captain of the new Winnipeg Jets franchise. - Sep 12, 2023. After a season without a captain, the Winnipeg Jets have named ...\nAdam Lowry, who has been a Jet since 2011 when he was drafted 67th overall, is the new captain of the NHL team — its third since relocating to Winnipeg from Atlanta in 2011. Andrew Ladd served ...\nWinnipeg played without a captain last season after stripping Blake Wheeler of the title in September. Lowry is entering his 10th season with the Jets, who drafted him in the third round in 2011.\nThe Winnipeg Jets will have a captain for the 2023-24 season. ... Although it will be his first time serving as a team captain since his final year with the Swift Current Broncos in 2012-13, Lowry ...'

Now we need to express how our function works in a way that is compatible with the OpenAI Function Calling API.

We'll want to provide a `JSON` object that includes what parameters we have, how to call them, and a short natural language description.

In [76]:
ddg_function = {
    "name" : "duckduckgo_search",
    "description" : "Answer non-technical questions. ",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "query" : {
                "type:" : "string",
                "description" : "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'"
            }
        },
        "required" : ["query"]
    }
}

####❓ Question

Why does the description key-value pair matter?

### Answer:

The description is used by the LLM to identify if the function needs to be called or not based on user query.

Now when we create our Assistant - we'll want to include the function description as a tool using the following format.

In [77]:
assistant = client.beta.assistants.create(
    name=name + " + Function Calling API",
    instructions=instructions,
    tools=[
        {"type": "function",
         "function" : ddg_function
        }
    ],
    model=model
)

We need to make a few modifications to our Assistant to include the ability to make calls to our local function and pass the results back to our Assistant for further generation.

In [78]:
import json

def wait_for_run_completion(thread_id, run_id):
    while True:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Current run status: {run.status}")
        if run.status in ['completed', 'failed', 'requires_action']:
            return run

def submit_tool_outputs(thread_id, run_id, tools_to_call):
    tool_output_array = []
    for tool in tools_to_call:
        output = None
        tool_call_id = tool.id
        function_name = tool.function.name
        function_args = tool.function.arguments

        if function_name == "duckduckgo_search":
            print("Consulting Duck Duck Go...")
            output = duckduckgo_search(query=json.loads(function_args)["query"])

        if output:
            tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

    print(tool_output_array)

    return client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_output_array
    )

def print_messages_from_thread(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    for msg in messages:
        print(f"{msg.role}: {msg.content[0].text.value}")

def use_assistant(query, assistant_id, thread_id=None):
  thread = client.beta.threads.create()

  message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role="user",
      content=query,
  )

  print("Creating Assistant ")

  run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
  )

  print("Querying OpenAI Assistant Thread.")

  run = wait_for_run_completion(thread.id, run.id)

  if run.status == 'requires_action':
    run = submit_tool_outputs(thread.id, run.id, run.required_action.submit_tool_outputs.tool_calls)
    run = wait_for_run_completion(thread.id, run.id)

  print_messages_from_thread(thread.id)

  return thread.id

####❓ Question

Outline, in simple terms, what the `use_assistant` helper function is doing.

### Answer:

use_assistant starts the execution of the LLM based on the prompt, instruction and available tools.


In [79]:
use_assistant("Who is James Marnie Argarin?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_pu1NJgOiSsK5lgb6Zvt5tqpc', 'output': 'About me. I am an enthusiastic and results-driven Software Engineer with a strong passion for continuous learning and professional growth within the field of software development. I am actively seeking an opportunity to apply my extensive skill set and knowledge to contribute meaningfully to the success of a forward-looking organization.\nStudied BS Computer Science at University of the Philippines Los Baños (UPLB) - itsybitsci\nTingnan ang profile ni James Marnie Argarin sa LinkedIn, isang propesyonal na komunidad na may 1 bilyong miyembro. I am an enthusiastic and results-driven Software Engineer with a strong passion for continuous learning and professional growth within the field of software development. I am actively seeking an opportunity to apply my extensive ...\nJAMES MARNIE N. ARGARIN Software Develo

'thread_VXlLAGLfpFtalBqZnIjGkAYZ'

In [81]:
use_assistant("At what University did James Marnie Argarin graduated from?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_JvGk6WYn1NSMalWiEj5v5Ats', 'output': 'James is a software engineer based in the Philippines with almost 2 years of professional work experience. ... Graduation. University of the Philippines Los Baños. Bachelor of Science in Computer Science; Cum Laude; University Scholar (2020-2023) DOST-SEI R.A. 7687 Scholar (2019-2023) August 2019 - August 2023 ...\nTingnan ang profile ni James Marnie Argarin sa LinkedIn, isang propesyonal na komunidad na may 1 bilyong miyembro. ... University of the Philippines Los Baños Ago 2023 Tingnan ang buong profile ni James Marnie Tingnan ang mga pareho ninyong kilala Maipakilala Direktang kontakin si James Marnie Sumali para makita ang buong profile ...\nCeremony Livestreams. Commencement ceremonies celebrating the Class of 2024 are available via livestream: Thursday, May 9th. 3:00 p.m. - The Graduate School. 6:30 p.m.

'thread_k4SvhSVHI83fvwWkFgyRasUM'

In [82]:
use_assistant("At what company does James Marnie Argarin work for at the moment?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_zFi3ZsChVZZ5cnrGeGxxdAky', 'output': 'Bachelor of Science in Computer Science; Cum Laude; University Scholar (2020-2023) DOST-SEI R.A. 7687 Scholar (2019-2023) August 2019 - August 2023\nTingnan ang profile ni James Marnie Argarin sa LinkedIn, isang propesyonal na komunidad na may 1 bilyong miyembro. I am an enthusiastic and results-driven Software Engineer with a strong passion for continuous learning and professional growth within the field of software development. I am actively seeking an opportunity to apply my extensive ...\nJAMES MARNIE N. ARGARIN Software Developer ⋄ (+63) 9694140079 jnargarin@gmail.com ⋄ LinkedIn ⋄ GitHub ⋄ Portfolio SUMMARY I am an enthusiastic and results-driven Software Developer with a strong passion for continuous learning and professional growth within the field of software development. I am actively seeking an oppor

'thread_jWL8DQl6SLqF39N6IWS0xqWB'

## Wrapping it All Together

Now we can create an Assistant with all of the available tools and see how it responds to various queries!

In [83]:
instructions

"You are an expert who will answer inquiries based on the attached file. If answer is not on the file, you can say 'I don't know.' Make sure your answer is correct by citing the source and explaining why it is correct."

In [84]:
instructions = "You are a helpful assistant. You will answer any questions asked of you based on the tools available."

In [85]:
assistant = client.beta.assistants.create(
    name=name + " + All Tools",
    instructions=instructions,
    tools=[
        {"type": "code_interpreter"},
        {"type": "file_search"},
        {"type": "function", "function" : ddg_function}
    ],
    tool_resources={
    "file_search": {
      "vector_store_ids": [vector_store.id]
    }
  },
    model=model,
)

In [86]:
use_assistant("Who is the current Captain of the Winnipeg Jets?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_JqRZsOoFomkxYjW9NeRXFshv', 'output': 'Winnipeg Jets Captains. Team Names: Winnipeg Jets, Atlanta Thrashers. Seasons: 24 (1999-00 to 2023-24) NHL Playoff Appearances: 8. NHL Championships: 0 (0 Stanley Cup) ... Current Summary/Standings, Current Schedule/Results, Current Leaders, Current Stats. 2023-24, ...\nLowry will follow Andrew Ladd and Blake Wheeler to serve as the third captain of the new Winnipeg Jets franchise. - Sep 12, 2023. After a season without a captain, the Winnipeg Jets have named ...\nLowry has been a part of the Jets organization since June 25, 2011, when he was selected in the third round, 67 th overall, after putting up 37 points in 36 games with the Swift Current Broncos of ...\nWinnipeg Jets. National team. Canada. NHL draft. 67th overall, 2011. Winnipeg Jets. Playing career. 2013-present. Adam Lowry (born March 29, 1993) is 

'thread_R4x9SHH0MAM7MwZxX0uppmZQ'

In [ ]:
use_assistant("Who is the author of the supplied file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The author of the supplied file titled "Frankenstein; or, The Modern Prometheus" is Mary Wollstonecraft Shelley【4:0†source】.
user: Who is the author of the supplied file?


'thread_srPpxHeU3MY4pAeeJF2vKyg4'

In [90]:
use_assistant("How many bytes is the provided file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The file "frankenstein.html" is approximately 15,647 bytes in size【4:0†source】.
user: How many bytes is the provided file?


'thread_FagkUfHiFCmFHH8a5fC14t3K'

####❓ Question

Notice that our response can go through multiple paths, given that:

What is "deciding" to use the tool?

### Answer:

- The "LLM" uses reasoning to decide which tool to use.

### Adding JSON Mode for More Agentic Behaviour

Finally, we have the ability to select tools - all we need to do now is set up a process to allow us to create some kind of loop and make decisions about whether or not the response is complete or not.

We'll leverage the OpenAI completions end-endpoint with JSON mode to let us understand when we've adequately answered our user's question!

In [91]:
completed_template = \
"""
Does this response adequately answer the user's query?

Please return your response in JSON format - with key: "completed" and either True (if completed) or False (if not completed)

User Query:
{query}

Assistant Response:
{response}
"""

def is_complete(query, response):
  completed_response = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": completed_template.format(query=query, response=response),
          }
      ],
      model=model,
      response_format={"type" : "json_object"}
  )

  return completed_response

In [98]:
query = "How many bytes is the provided file?"

thread_id_for_response = use_assistant(query, assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The file "frankenstein.html" contains approximately 1221 bytes of text content.
assistant: The file "frankenstein.html" contains text in UTF-8 encoding. The exact size of the file in bytes can be determined by analyzing the text content. Let's calculate the size of the file.
user: How many bytes is the provided file?


Now we can observe JSON mode in action!

In [99]:
messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)
response = messages.data[0].content[0].text.value
completed_flag = json.loads(is_complete(query, response).choices[0].message.content)

In [100]:
completed_flag

{'completed': True}

## 🚧 BONUS CHALLENGE 🚧:

Use the components we've constructed so far to build a loop that lets us continue to query the Assistant if the response is not completed!

In [109]:
query = "What happened at the end of Frankenstein?"

thread_id_for_response = use_assistant(query, assistant.id)

iteration = 0
completed_flag = False
while completed_flag == False:
  messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)
  response = messages.data[0].content[0].text.value
  completed_flag = json.loads(is_complete(query, response).choices[0].message.content)
  print(completed_flag)
  print(iteration)
  iteration += 1 

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: At the end of "Frankenstein," the creature reflects on the misery and despair that led to his actions and the consequences of his choices. The story concludes with the creature expressing remorse and reflecting on the destruction and pain caused by his actions, as well as the impact on the protagonist, Victor Frankenstein. The creature ultimately seeks understanding and compassion from Frankenstein, highlighting the complex moral and emotional themes explored in the novel.

For a detailed summary of the conclusion of "Frankenstein," you can refer to the document "frankenstein.html" that provides a description of the final events and character interactions【4:0†source】【4:1†source】【4:2†source】【4:3†source】.
user: What happened at the end of Frankenstein?
{'completed': True}
0


# Make Sure You Delete Resources

Make sure you delete all the resources you created!

This function will help you do so!

In [111]:
file_deletion_status = client.beta.assistants.delete(
  assistant_id=assistant.id,
)